# Data analysis using LLM models 

### reading the essential packages

In [62]:

import sys
# !{sys.executable} -m pip install langchain
# !{sys.executable} -m pip install langchain_community
# !{sys.executable} -m pip install pandasai
# !{sys.executable} -m pip install ollama
# !{sys.executable} -m pip install chromadb
# !{sys.executable} -m pip install pysqlite3-binary
!{sys.executable} -m pip install -U langchain-nomic 
!{sys.executable} -m pip install -U tiktoken 
!{sys.executable} -m pip install -U langchainhub 
!{sys.executable} -m pip install -U langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 72.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for nomic: filename=nomic-3.0.25-py3-none-any.whl size=44318 sha256=96e6f6f34e93f0ce8721a07fbc9ddd21321b66f0c0545f1721a41d8bd7a13924
  Stored in directory: /home/ubuntu/.cache/pip/wheels/48/3e/81/caa78ba4afad07dd8e51f8e54384279ff8f590a17c403a1e2d
Successfully built nomic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.2/785.2 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43

In [1]:
import pandas as pd  
from pandasai import SmartDataframe
from pandasai import Agent
# Instantiate a LLM
from pandasai.llm import OpenAI
from pandasai.llm.local_llm import LocalLLM
from langchain_community.llms import Ollama
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import dataframe
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# Reading the data 
data = pd.read_csv('../data/population.csv')
data.sample(n=10)

,Country,Population
10,Bangladesh,172954319
16,DR Congo,102262808
12,Mexico,128455567
0,United States,339996563
5,Indonesia,277534122
3,Turkey,85816199
18,Iran,89172767
1,China,1425671352
19,Thailand,71801279
7,Nigeria,223804632


In [24]:
# init an llm model
# llm = LocalLLM(api_base="http://localhost:11434/v1",model='llama3')
gpt4_llm = OpenAI(api_token="OPENAIKEY",model="gpt-4")

llm = Ollama(model = 'llama3')
sdf = SmartDataframe(data, config={"llm":gpt4_llm})

In [25]:
sdf.chat('Which country which has max population and what is the value?')

,Country,Population,doc
8,India,1428627663,country: India population: str(1428627663)


In [26]:
print(sdf.last_code_generated)

max_population_country = dfs[0][dfs[0]['Population'] == dfs[0]['Population'].max()]
result = {'type': 'dataframe', 'value': max_population_country}


In [60]:
sdf.chat("what is the popluation of India? ")

1428627663

In [61]:
print(sdf.last_code_generated)

population_india = None
for df in dfs:
    if 'India' in df['Country'].values:
        population_india = df.loc[df['Country'] == 'India', 'Population'].values[0]
        break
result = {'type': 'number', 'value': population_india}


# RAG system

In [12]:
def create_doc (country, population):
    return f"country:  {country}  population: str({population })"

create_doc('india',3444444) 

'country:  india  population: str(3444444)'

In [13]:
data['doc'] = data.apply(lambda x: create_doc(x['Country'],x['Population']),axis=1)

In [29]:
docs = dataframe.DataFrameLoader(data,page_content_column='doc').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [30]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [31]:
# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context} , give the answer with respect to context"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

In [33]:
# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context) , retrieved_docs

In [56]:
result , evm_docs =rag_chain("the population str of country Russia?")

In [57]:
print(result)


I see what you're getting at!

You want me to extract the population figures from the given contexts, right?

Alright, let's do that:

**Russia**: The population of Russia is approximately **144.44 million** (as per your context).

**Japan**: And, according to your context, the population of Japan is approximately **12.32945 million**.

Please note that these figures might not be up-to-date or accurate, as they are based on the provided contexts and not actual census data.


In [58]:
print(evm_docs)

[Document(page_content='country:  Russia  population: str(144444359)', metadata={'Country': 'Russia', 'Population': 144444359}), Document(page_content='country:  Russia  population: str(144444359)', metadata={'Country': 'Russia', 'Population': 144444359}), Document(page_content='country:  Japan  population: str(123294513)', metadata={'Country': 'Japan', 'Population': 123294513}), Document(page_content='country:  Japan  population: str(123294513)', metadata={'Country': 'Japan', 'Population': 123294513})]
